In [1]:
!pip install deepface
!pip install tensorflow
# from deepface import DeepFace
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=b08eeffc088d86b6a6f31608ca3c94151dc01d72b9bdb8e6f535e8f350a94ec4
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
25-03-14 01:23:02 - Directory /root/.deepface has been created
25-03-14 01:23:02 - Directory /root/.deepface/weights has been created


In [1]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Test of Accuracy on the FER2013 Dataset

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("astraszab/facial-expression-dataset-image-folders-fer2013")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/astraszab/facial-expression-dataset-image-folders-fer2013/versions/1


In [4]:
from pathlib import Path
import pandas as pd

def get_all_png_paths(root_folder):
    """读取指定文件夹及其子文件夹中的所有PNG图像文件路径"""
    root_path = Path(root_folder)

    all_png_paths = []
    folder_images = {}

    for folder_idx in range(7):
        folder_name = str(folder_idx)
        folder_path = root_path / folder_name

        if not folder_path.exists() or not folder_path.is_dir():
            print(f"not found {folder_path}")
            continue

        png_paths = [str(p) for p in folder_path.glob("**/*")]
        png_paths = [p for p in png_paths if Path(p).is_file()]

        all_png_paths.extend(png_paths)
        folder_images[folder_name] = png_paths
    return all_png_paths, folder_images

test_folder = path+"/data/test"
all_images, images_by_folder = get_all_png_paths(test_folder)

# 创建DataFrame
data = []
for folder_name, image_paths in images_by_folder.items():
    for path in image_paths:
        data.append({
            'image_path': path,
            'emotion': int(folder_name)
        })

image_df = pd.DataFrame(data)
print(image_df.head())

                                          image_path  emotion
0  /root/.cache/kagglehub/datasets/astraszab/faci...        0
1  /root/.cache/kagglehub/datasets/astraszab/faci...        0
2  /root/.cache/kagglehub/datasets/astraszab/faci...        0
3  /root/.cache/kagglehub/datasets/astraszab/faci...        0
4  /root/.cache/kagglehub/datasets/astraszab/faci...        0


In [5]:
emotion_mapping = {
    0: 'angry',
    1: 'disgust',
    2: 'fear',
    3: 'happy',
    4: 'sad',
    5: 'surprise',
    6: 'neutral'
}
def detect_emotion(data,data_emotion,detector_backend='opencv'):
    accuracy = 0
    detected_emotions=[]
    for idx,img in tqdm(enumerate(data), total=len(data), desc="Analyzing emotions"):
        result = DeepFace.analyze(img, detector_backend=detector_backend,enforce_detection=False, silent = True)
        # print(result)
        emotion = result[0]['dominant_emotion']
        detected_emotions.append((idx,emotion))
        # print(emotion,data_emotion[idx])
        if emotion == emotion_mapping[data_emotion[idx]]:
            accuracy += 1
    return accuracy/len(data),detected_emotions



In [14]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.6/932.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [6]:
backends = [
    'opencv', 'ssd', 'dlib', 'mtcnn', 'fastmtcnn',
    'retinaface', 'mediapipe', 'yolov8', 'yolov11s',
    'yolov11n', 'yolov11m', 'yunet', 'centerface',
]
align = True

# 获取emotion标签和pixels数据
y_test=image_df['emotion'].values
x_test=image_df['image_path'].values
print(f'Number of test images: {len(x_test)}')

# accuracy, detected_emotions = detect_emotion(x_test,y_test,detector_backend='dlib')
# print(f'Accuracy with {i} backend: {accuracy:.2%}')

for i in backends:
    accuracy, detected_emotions = detect_emotion(x_test,y_test,detector_backend=i)
    print(f'Accuracy with {i} backend: {accuracy:.2%}')


Number of test images: 3589


Analyzing emotions:   0%|          | 0/3589 [00:00<?, ?it/s]


InvalidArgumentError: Exception encountered when calling layer 'conv2d' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} No DNN in stream executor. [Op:Conv2D] name: 

Call arguments received by layer 'conv2d' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 48, 48, 1), dtype=float32)